In [1]:
import tensorflow as tf

/home/phunc20/.config/miniconda3/envs/homl1e/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/phunc20/.config/miniconda3/envs/homl1e/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/phunc20/.config/miniconda3/envs/homl1e/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/phunc20/.c

In [2]:
n_inputs = 3   # This is the k in recurrent-neurons.pdf
n_neurons = 5  # This is the l in recurrent-neurons.pdf

### RNN manually

In [3]:
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

#Wx = tf.Variable(tf.random_normal(shape=[k,l], dtype=tf.float32))
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
#Wy = tf.Variable(tf.random_normal(shape=[l,l], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
#b = tf.Variable(tf.zeros(shape=[1,l], dtype=tf.float32))
b = tf.Variable(tf.zeros(shape=[1, n_neurons], dtype=tf.float32))

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
Y0 = tf.tanh(X0 @ Wx + b)
#Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(X1 @ Wx + Y0 @ Wy + b)
#Y1 = tf.tanh(tf.matmul(X1, Wx) + tf.matmul(Y0, Wy) + b)

In [5]:
Y0, Y1

(<tf.Tensor 'Tanh:0' shape=(?, 5) dtype=float32>,
 <tf.Tensor 'Tanh_1:0' shape=(?, 5) dtype=float32>)

In [6]:
init = tf.global_variables_initializer()

In [7]:
import numpy as np
# batch size: 4
# mini-batch:        instance0, instance1, instance2, instance3
X0_batch = np.array([[0,1,2],   [3,4,5],   [6,7,8],   [9,0,1],])  # t = 0
X1_batch = np.array([[9,8,7],   [0,0,0],   [6,5,4],   [3,2,1],])  # t = 1

In [8]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [11]:
print(f"Y0_val =\n{Y0_val}")
print()
print(f"Y1_val =\n{Y1_val}")

Y0_val =
[[-0.9904115  -0.99931055  0.6659969   0.9933008  -0.82816523]
 [-1.         -1.         -0.9993631   1.          0.9887699 ]
 [-1.         -1.         -0.99999994  1.          0.99999404]
 [-1.          0.35226482 -1.          1.          1.        ]]

Y1_val =
[[-1.         -1.         -1.          1.          1.        ]
 [-0.7602829  -0.6828733   0.98884517  0.4828381   0.99999297]
 [-1.         -1.         -0.9999997   1.          1.        ]
 [-0.9999988  -0.99993604 -0.9997792   0.99985635  1.        ]]


The output numerical values like above have little meaning in this notebook, so don't pay too much attention to them.

### `static_rnn()`
The `static_rnn()` function creates an unrolled RNN network by chaining cells. The following
code creates the exact same model as the previous one.

In [17]:
tf.reset_default_graph()

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1], dtype=tf.float32)
Y0, Y1 = output_seqs

init = tf.global_variables_initializer()

In [18]:
with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [19]:
print(f"Y0_val =\n{Y0_val}")
print()
print(f"Y1_val =\n{Y1_val}")

Y0_val =
[[-0.2637325   0.4074781   0.59535986 -0.1350736   0.83629495]
 [-0.9872699  -0.73377454  0.9968244  -0.6706742   0.9810802 ]
 [-0.9998592  -0.98034304  0.99998003 -0.9029616   0.9979559 ]
 [-0.9998874  -0.9988469   0.99991506 -0.6895312  -0.9998323 ]]

Y1_val =
[[-0.9999928  -0.9996412   0.9999988  -0.92684174  0.9397128 ]
 [ 0.7526528   0.8308104   0.6872208   0.70348865  0.11825394]
 [-0.9959175  -0.958328    0.99994886 -0.20495862  0.86012644]
 [-0.94188434 -0.38022158  0.982755    0.46705097  0.36510664]]


What was shown next on the book was the use of
- `tf.unstack()`
  - `tf.stack()`
- `tf.transpose()`
to allow input of a different (but equivalent) shape.

Note that the code and shape of tensors in `BasicRNNCell()` and in `static_rnn()` remains identical
to previously.

In [22]:
n_steps = 2
tf.reset_default_graph()
X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1,0,2]))
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs, dtype=tf.float32)
# At this stage, shape(output_seqs) = (n_steps, None, n_inputs) (or at least approximately)
# But we'd like to have sth of shape (None, n_steps, n_inputs)
outputs = tf.transpose(tf.stack(output_seqs), perm=[1,0,2])

In [23]:
init = tf.global_variables_initializer()
X_batch = np.array([
    # t = 0
    [[0,1,2], [9,8,7]],
    [[3,4,5], [0,0,0]],
    [[6,7,8], [6,5,4]],
    [[9,0,1], [3,2,1]],
])

In [24]:
sess = tf.InteractiveSession()
init.run()
outputs_val = outputs.eval(feed_dict={X: X_batch})
sess.close()
outputs_val

array([[[ 0.8824959 ,  0.47511262, -0.72453445, -0.5547652 ,
          0.8765556 ],
        [ 0.9999982 ,  0.9980418 ,  0.27721155,  0.00250062,
          0.99999994]],

       [[ 0.99962145,  0.9369851 , -0.7670588 , -0.454684  ,
          0.9998627 ],
        [-0.5213208 ,  0.28629366, -0.29050636, -0.7909661 ,
         -0.02074875]],

       [[ 0.9999988 ,  0.9940689 , -0.80376554, -0.34163246,
          0.9999999 ],
        [ 0.9989311 ,  0.9724046 ,  0.4400794 , -0.14415829,
          0.9999684 ]],

       [[ 0.03162497, -0.37233666,  0.9994967 ,  0.99834436,
          0.9964416 ],
        [ 0.86975074,  0.6934972 ,  0.76061445,  0.716446  ,
          0.9819747 ]]], dtype=float32)

### OOM (Out-of-memory) error
**(?)** I don't quite understand oom here. Does it mean that `dynamic_rnn()` won't suffer from oom?